In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import holodeck.detstats as ds
import h5py

import numpy as np
import matplotlib.pyplot as plt

# Calculate Library Detstats

In [ ]:
# hdf_name = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/ss_lib.hdf5'
# detstats_path = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/detstats'

# npsrs = 40
# sigma = 1e-7
# output_dir = detstats_path+'/ds2_psrs40_sigma1e-7'

In [ ]:
# vals = ds.detect_lib(hdf_name, output_dir, npsrs, sigma, 
#                      nskies=25, thresh=0.5, plot=True, debug=True)

# Read in Library Detstats

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0/'
hdfname = sspath+'ss_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][...]
hc_bg = ssfile['hc_bg'][...]
ssfile.close()

print('N,F,R,L:', hc_ss.shape) 
shape = hc_ss.shape
N, F, R, L = shape[0], shape[1], shape[2], shape[3]

# dsfile = np.load('/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0/detstats/dp4_psrs60_sigma1e-07/detstats.npz')
dsfile = np.load('/Users/emigardiner/GWs/holodeck/output/2023-05-12-mbp-ss16_n10_r10_f70_d12.5_l10_p0/detstats/ds2_psrs60_sigma5e-07/detstats.npz')
dp_ss = dsfile['dp_ss']
dp_bg = dsfile['dp_bg']
df_ss = dsfile['df_ss']
df_bg = dsfile['df_bg']
snr_ss = dsfile['snr_ss']
snr_bg = dsfile['snr_bg']
print('N,R,S:', dp_ss.shape)
S = dp_ss.shape[-1]

In [ ]:
print('snr_ss range:', np.min(snr_ss), np.max(snr_ss))

# Integration Outputs

In [ ]:
from scipy import integrate, special
from sympy import nsolve, Symbol

def _Fe_thresh(Num, alpha_0=0.001):
    """ Calculate the threshold F_e statistic using sympy.nsolve
    
    Parameters
    ----------
    Num : int
        Number of single sources to detect.
    alpha_0 : scalar
        False alarm probability max.

    Returns
    -------
    Fe_bar : scalar
        Threshold Fe statistic
    """
    Fe_bar = Symbol('Fe_bar')
    func = 1 - (1 - (1 + Fe_bar)*np.e**(-Fe_bar))**Num - alpha_0
    Fe_bar = nsolve(func, Fe_bar, 10)
    return(Fe_bar)

def _integrand(Fe, rho):

    I_1 = special.i1(rho*np.sqrt(2*Fe))
    rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv

def integrate_rho(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    
    Parameters
    ----------
    rho : scalar
    Fe_bar : scalar

    Returns
    -------
    gamma) : float
    """
    gamma_ssi = integrate.quad(_integrand, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

In [ ]:
Num = F*L
Fe_bar = _Fe_thresh(Num)
outname = './output/integral_maps/Num%d_F%d_L%d_Febar%.2f.dat' % (Num, F, L, Fe_bar)

In [ ]:
rho = np.geomspace(10**-3, 10**3, 10**5)
print(rho.shape)
ans = np.zeros_like(rho)


for ii in range(len(rho)):
    if ii%(len(rho)/10) == 0:
       print(ii, 'out of', len(rho))
    ans[ii] = integrate_rho(Fe_bar, rho[ii])


In [ ]:
xx = rho
yy = ans

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
ax1.loglog(xx, yy, label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.legend()

ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)

fig.tight_layout()

In [ ]:
rho1 = np.geomspace(10**0.5, 10**2, 10**3)
print(rho1.shape)
ans1 = np.zeros_like(rho1)


for ii in range(len(rho1)):
    if ii%(len(rho1)/10) == 0:
       print('on', ii, 'out of', len(rho1))
    ans1[ii] = integrate_rho(Fe_bar, rho1[ii])

In [ ]:
xx=rho1
yy=ans1
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
ax1.loglog(xx, yy, label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.legend()

ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)

fig.tight_layout()

In [ ]:
print(snr_ss.shape, snr_bg.shape)

In [ ]:
plt.hist(snr_ss.flatten(), bins=100)
plt.yscale('log')
plt.xscale('log')
plt.title('Histogram of SNRs for the 10th Loudest Sources in each Frequency Bin')
plt.ylabel('Number of Single Sources')
plt.xlabel('SNR_ss')

In [ ]:
print(Fe_bar)
Fe = np.linspace(start=15, stop=1000, num=10)
integrands = np.zeros((len(Fe), len(rho)))
I1 = np.zeros_like(integrands)
for ff in range(len(Fe)):
    if ff%(len(Fe)/10) == 0:
        print('on', ff, 'out of', len(Fe))
        for ii in range(len(rho)):
            integrands[ff,ii] = _integrand(Fe=Fe[ff], rho=rho[ii])
            I1[ff,ii] = special.i1(rho[ii]*np.sqrt(2*Fe[ff]))

In [ ]:
import matplotlib.cm as cm

In [ ]:

# for ff in range(len(Fe)):
#     plt.loglog(rho, integrands[ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
# plt.legend(loc='lower left')
# plt.xlabel(r'$\rho$')
# plt.ylabel('integrand')
# plt.plot(np.arange(len(Fe)), Fe)

xx=rho
yy=integrands
colors = cm.rainbow(np.linspace(0,1,len(Fe)))
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
for ff in range(len(Fe)):
    ax1.loglog(rho, yy[ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
    ax2.scatter(xx, np.isnan(yy[ff]), marker='x', s=1, color=colors[ff], alpha=0.5)

# ax1.loglog(xx, yy[0], label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$', 
               fontsize=12)
ax1.set_ylabel('integrand')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='integrand=1')
ax1.legend()


ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)

fig.tight_layout()

In [ ]:

# for ff in range(len(Fe)):
#     plt.loglog(rho, integrands[ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
# plt.legend(loc='lower left')
# plt.xlabel(r'$\rho$')
# plt.ylabel('integrand')
# plt.plot(np.arange(len(Fe)), Fe)

xx=rho
yy=I1
colors = cm.rainbow(np.linspace(0,1,len(Fe)))
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1, 1]})
for ff in range(len(Fe)):
    ax1.loglog(rho, yy[ff]/10**50, color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
    ax2.scatter(xx, np.isnan(yy[ff]), marker='x', s=1, color=colors[ff], alpha=0.5)
    ax3.scatter(xx, np.isinf(yy[ff]), marker='x', s=1, color=colors[ff], alpha=0.5)

# ax1.loglog(xx, yy[0], label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'$\mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) $', 
               fontsize=12)
ax1.set_ylabel(r'$\mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) /10^{50}$')
# ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.legend()
ax1.set_ylim(10**-54, 10**19) # it won't let me set ymax above 10^19


ax2.set_ylabel('is nan')
ax2.set_ylim(-0.1,1.1)
ax3.set_ylabel('is inf')
ax3.set_ylim(-0.1,1.1)
ax3.set_xlabel(r'$\rho$', fontsize=14)

fig.tight_layout()

In [ ]:
print(I1[np.isinf(I1)].size)
print(I1.size)

## Try again, using long for I1

In [ ]:
import sys
print(sys.float_info.max)
print(np.max(I1[np.isfinite(I1)]))
print(sys.dou

In [ ]:
# print(Fe_bar)
# Fe = np.linspace(start=15, stop=1000, num=10)
# integrands = np.zeros((len(Fe), len(rho)))
I1_long = np.zeros_like(integrands, dtype=long)
for ff in range(len(Fe)):
    if ff%(len(Fe)/10) == 0:
        print('on', ff, 'out of', len(Fe))
        for ii in range(len(rho)):
            I1_long[ff,ii] = special.i1(rho[ii]*np.sqrt(2*Fe[ff]))

## Try setting inf values to large I1

In [ ]:

def _integrand_max(Fe, rho):

    I_1 = special.i1(rho*np.sqrt(2*Fe))
    if np.isinf(I_1): I_1 = 10**300
    rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv

def integrate_rho_max(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    
    Parameters
    ----------
    rho : scalar
    Fe_bar : scalar

    Returns
    -------
    gamma) : float
    """
    gamma_ssi = integrate.quad(_integrand_max, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

In [ ]:
print(Fe_bar)
Fe = np.linspace(start=15, stop=1000, num=10)
integrands_max = np.zeros((len(Fe), len(rho)))
I1_max = np.zeros_like(integrands)
ans_max = np.zeros_like(rho)

for ii in range(len(rho)):
    ans_max[ii] = integrate_rho_max(Fe_bar, rho[ii])
    if ii%(len(rho)/10) == 0:
        print('on', ii, 'out of', len(rho))
        for ff in range(len(Fe)):
            integrands_max[ff,ii] = _integrand(Fe=Fe[ff], rho=rho[ii])
            I1_max[ff,ii] = special.i1(rho[ii]*np.sqrt(2*Fe[ff]))

In [ ]:
print('max gamma = gamma[rho=%.2f] = %.10f' % (rho[np.argmax(ans_max)], np.max(ans_max)))
index_one = 0
ii=0
while index_one==0:
    if(ans_max[ii]>=1): index_one = ii
    ii+=1
print('First gamma>=1 = gamma(rho=%.2f) =  %.10f' % (rho[index_one], ans_max[index_one]))

# print(ans_max[index_one], ans_max[index_one+1], ans_max[index_one+2])
# for ii in range(index_one, len(ans_max)-1):
#     if(ans_max[ii]<ans_max[ii+1]):
#         print(ii, 'increasing')
#     elif((ans_max[ii]>ans_max[ii+1])):
#         print(ii,'decreasing')

deriv = np.diff(ans_max)/ans_max[1:]
plt.scatter(rho[1:], deriv>0)
plt.xscale('log')
plt.axvline(rho[index_one], color='k', linestyle='dashed')



In [ ]:
xx = rho
yy = ans_max

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
ax1.loglog(xx, yy, label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.axvline(rho[index_one], label=r'$\gamma_i(\rho=%.2f) = 1$' % rho[index_one], color='green', linestyle='--')
ax1.legend()

ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)


fig.tight_layout()

# Integrate with Euler Method
and bessel function max patch

F(t, S(t)) = gamma(Fe, integrand(Fe)). For some fixed rho
<!-- 
$\frac{dS(t)}{dt} = \mathrm{integrand}(\mathcal{F}_e) = \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} = F(t, S(t))$

$S(t) = \gamma_i(\rho) = \int \frac{dS(\rho)}{d\rho} d\rho$

For my code:

$F(\rho) = \frac{d\gamma(\rho)}{d\rho} = \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$

$ \gamma(\rho_{j+1}) = \gamma(\rho_j) + (\rho_{j+1} - \rho_j) \frac{d\gamma(\rho_j)}{dt}$ -->

$\rho$ = fixed

$\frac{dS(t)}{dt} = \frac{\sqrt{2t}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2t} ) e^{-t - \frac{1}{2}\rho^2}$ given by _integrand_max(t, rho)

$\frac{dS(t)}{dt} = F(t, S(t))$

$s1 = s(t1) = s(0) = \int \frac{dS(t)}{dt} dt $ at $t=0$

In [ ]:
# Define parameters
def f(t, rho):
    return _integrand_max(t, rho)

h = 0.0001 # step size
t = np.arange(0, 1+h, h) # numerical grid
s0 = -1 # initial condition, but not sure what this value should actually be, 
# but maybe it doesn't matter because it's just a constant that would be subtracted in finite integral subtraction


s=np.zeros(len(t))
s[0] = s0
print(s)

rr = int(len(rho)/2) # pick a middleish rho
print(rho[rr])
for i in range(0, len(t)-1):
    s[i+1] = s[i] + h*f(t[i], rho[rr])
gamma1= s[-1] - s[0]

plt.scatter(t, s, marker='x', label='rho=%f, gamma(rho)=%f' % (rho[rr], gamma1), s=3)
plt.legend()
plt.xlabel('t')
plt.ylabel('f(t)')

In [ ]:
# Want integral from Fe_bar to infinity

# Define parameters
def f(t, rho):
    return _integrand_max(t, rho)
print(Fe_bar)
h = 1 # step size
t = np.arange(float(Fe_bar),float(Fe_bar)*10000, h) # numerical grid
s0 = 0 # initial condition, not sure what this value should actually be, 
# but it doesn't matter because it's just a constant subtracted out of definite integrals

s=np.zeros(len(t))
s[0] = s0
# print(s)

rr = int(len(rho)/2) # pick a middleish rho
print(rho[rr])
for i in range(0, len(t)-1):
    s[i+1] = s[i] + h*f(t[i], rho[rr])
gamma1= s[-1] - s[0]

plt.scatter(t, s, marker='x', label='rho=%f, gamma(rho)=%f' % (rho[rr], gamma1), s=3)
plt.legend()
plt.xlabel('t')
plt.ylabel('f(t)')

In [ ]:
# Define parameters
def f(t, rho):
    return _integrand_max(t, rho)
h = 10 # step size
t = np.arange(float(Fe_bar), float(Fe_bar)*100, h) # numerical grid
s0 = 0 # it doesn't matter what this is because it's just subtracted out for definite integrals
s=np.zeros(len(t))

gamma = np.zeros_like(rho)
for rr in range(len(rho)):
    if rr%(len(rho)/10) == 0: print('on', rr, 'out of', len(rho))
    for ii in range(0, len(t)-1):
        s[ii+1] = s[ii] + h*f(t[ii], rho[rr])
    gamma[rr] = s[-1] - s[0]

In [ ]:
h=10
plt.loglog(rho, gamma, marker='x', markersize=1, linestyle='', alpha=0.5,
           label = 'Euler Method with h = %d from $\overline{F}_e$ to 100$\overline{F}_e$' % h)
plt.loglog(rho, ans_max, marker='x', markersize=1, linestyle='',  alpha=0.5,
           label = 'Scipy Integration')
plt.xlabel('rho')
plt.ylabel('gamma')
plt.axhline(1, label='gamma=1', linestyle='--', color='k')
plt.legend(loc='lower left')

In [ ]:

plt.loglog(rho, gamma, marker='x', markersize=3, linestyle='')
plt.xlabel('rho')
plt.ylabel('gamma')
plt.title(r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$ by Euler Method', 
               fontsize=12)
plt.xlim(10**-3, 10**0)
plt.ylim(10**-1, 10)

In [ ]:
rho3 = np.geomspace(10**-3, 10**1, 100)

# Define parameters
def f(t, rho):
    return _integrand_max(t, rho)

h = 1 # step size
t = np.arange(float(Fe_bar), float(Fe_bar)*1000, h) # numerical grid from Fe_bar to ~inf (or just large)
s0 = 0 # it doesn't matter what this is because it's just subtracted out in finite integral subtraction
s=np.zeros(len(t))

xx = rho3
y1 = np.zeros_like(rho3)
y2 = np.zeros_like(rho3)
for rr in range(len(xx)):
    if rr%(len(xx)/10) == 0: print('on', rr, 'out of', len(xx))
    for ii in range(0, len(t)-1):
        s[ii+1] = s[ii] + h*f(t[ii], xx[rr])
    y1[rr] = s[-1] - s[0]
    y2[rr] = integrate_rho(Fe_bar, xx[rr])
gamma3 = y1
ans3 = y2

In [ ]:

plt.loglog(xx, y1, marker='x', markersize=3, linestyle='', label='Euler Method with h = %.3f from $\overline{F}_e$ to $1000\overline{F}_e$' % h)
plt.loglog(xx, y2, marker='x', markersize=3, linestyle='', label='Scipy Integration')
plt.xlabel('rho')
plt.ylabel('gamma')
plt.legend(loc='upper left')

# Try integrating with quadpy
## without extra pars
Setting rho outside of the function, so the integrand is a function only of Fe.

Bessel function fails??

In [ ]:
import quadpy

In [ ]:
def _integrand_qp(Fe):
        rho_qp=rho2[5]
        I_1 = special.i1(rho_qp*np.sqrt(2*Fe))
        rv = (2*Fe)**(1/2) /rho_qp * I_1 * np.exp(-Fe - rho_qp**2 /2)
        return rv

def integrate_rho_quadpy(Fe_bar):
    """ Calculate the detection probability for each single source in each realization.
    
    Parameters
    ----------
    rho : scalar
    Fe_bar : scalar

    Returns
    -------
    rv : float
    """
    gamma_ssi = quadpy.quad(_integrand_qp, Fe_bar, np.inf)[0]
    return gamma_ssi

In [ ]:
rho2 = np.geomspace(10**0.5, 10**2, 10**3)
print(rho1.shape)
ans2 = np.zeros_like(rho1)

for ii in range(len(rho2)):
    if ii%(len(rho2)/10) == 0:
       print('on', ii, 'out of', len(rho2))
    rho_qp=rho2[ii]  
    print(rho_qp)
    ans2[ii] = integrate_rho_quadpy(Fe_bar)

## same way as with integrand.py
This doesn't work

In [ ]:
def _integrand_qp(Fe, rho):

    I_1 = special.i1(rho*np.sqrt(2*Fe))
    rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv

def integrate_rho_quadpy(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    
    Parameters
    ----------
    rho : scalar
    Fe_bar : scalar

    Returns
    -------
    rv : float
    """
    gamma_ssi = quadpy.quad(_integrand, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

In [ ]:
rho2 = np.geomspace(10**0.5, 10**2, 10**3)
print(rho1.shape)
ans2 = np.zeros_like(rho1)

for ii in range(len(rho2)):
    if ii%(len(rho2)/10) == 0:
       print('on', ii, 'out of', len(rho2))
    ans2[ii] = integrate_rho_quadpy(Fe_bar, rho2[ii])